# Detector de matriculas

In [10]:
import cv2
import pytesseract
import re
from ultralytics import YOLO
from pytesseract import Output
import easyocr
import math

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 608x640 1 car, 1 truck, 213.9ms
Speed: 5.0ms preprocess, 213.9ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 4 persons, 1 car, 217.0ms
Speed: 5.0ms preprocess, 217.0ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 4 persons, 1 car, 174.6ms
Speed: 4.7ms preprocess, 174.6ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 person, 1 car, 209.2ms
Speed: 4.2ms preprocess, 209.2ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 192.5ms
Speed: 3.0ms preprocess, 192.5ms inference, 1.1ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 204.2ms
Speed: 4.0ms preprocess, 204.2ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 196.9ms
Speed: 3.4ms preprocess, 196.9ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 192.0ms
Speed: 4.5ms preprocess, 192.0ms inference, 2.0ms postpro

: 

In [2]:
from inference_sdk import InferenceHTTPClient
from collections import defaultdict
import base64
import cv2
import easyocr
import math
from ultralytics import YOLO

reader = easyocr.Reader(['en'])  # Inicializamos el lector de easyocr
model = YOLO('modelo.pt')  # Carga del modelo YOLOv8 nano para detección

CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="wwc5TpbiPDQT2hBThzFI"
)

# Etiquetas de clases relevantes
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]

video_path = 'video.mp4'
vid = cv2.VideoCapture(video_path)
track_history = defaultdict(lambda: [])

# Función para codificar imagen a base64
def encode_image_to_base64(image):
    _, buffer = cv2.imencode('.jpg', image)
    jpg_as_text = base64.b64encode(buffer).decode('utf-8')
    return jpg_as_text

# Función para detectar matrícula solo si no se ha procesado antes
def detect_license_plate(plate_img):
    encoded_image = encode_image_to_base64(plate_img)
    response = CLIENT.infer(encoded_image, model_id="license-plate-ojt1x/1")
    predictions = response.get("predictions", [])

    for prediction in predictions:
        x1 = int(prediction['x'] - prediction['width'] / 2)
        y1 = int(prediction['y'] - prediction['height'] / 2)
        x2 = int(prediction['x'] + prediction['width'] / 2)
        y2 = int(prediction['y'] + prediction['height'] / 2)
        confidence = round(prediction['confidence'], 2)

        print(f"Confianza ---> {confidence}")
        print(f"Clase ---> {prediction.get('class', '')}")

        # Dibujar el contenedor y clase
        cv2.rectangle(plate_img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(plate_img, f"Matricula {confidence}", (x1, y1 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

while True:
    ret, img = vid.read()

    if ret:
        img = cv2.resize(img, (640, 600))  # Reducimos resolución
        results = model.track(img, persist=True, classes=[0, 1, 2, 3, 5, 7])

        for r in results:
            boxes = r.boxes

            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                if box.id is not None:
                    track_id = str(int(box.id[0].tolist()))
                else:
                    track_id = ''

                confidence = math.ceil((box.conf[0] * 100)) / 100
                cls = int(box.cls[0])
                class_name = classNames[cls] if cls < len(classNames) else "unknown"

                color = (0, 255, 0) if cls == 0 else (0, 0, 255)  # Verde para personas, rojo para vehículos
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, f"{track_id} {class_name} {confidence:.2f}", 
                            (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                # Si es un vehículo, realizar la detección de matrícula
                if cls in [2, 3, 5, 7]:  # Solo vehículos
                    if track_id not in track_history:  # Solo detecta si es un vehículo nuevo
                        plate_img = img[int(y1 + (y2 - y1) * 0.6):y2, x1:x2]
                        detect_license_plate(plate_img)
                        track_history[track_id].append("processed")

        cv2.imshow('Detección y super seguimiento', img)
    else:
        break

    if cv2.waitKey(20) == 27:
        break

vid.release()
cv2.destroyAllWindows()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 608x640 3 plates, 263.6ms
Speed: 7.2ms preprocess, 263.6ms inference, 13.1ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 plate, 211.8ms
Speed: 4.0ms preprocess, 211.8ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 plate, 236.6ms
Speed: 4.1ms preprocess, 236.6ms inference, 3.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 plate, 192.8ms
Speed: 2.5ms preprocess, 192.8ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 plate, 265.0ms
Speed: 6.1ms preprocess, 265.0ms inference, 1.1ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 (no detections), 224.6ms
Speed: 6.1ms preprocess, 224.6ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 (no detections), 225.5ms
Speed: 3.0ms preprocess, 225.5ms inference, 0.6ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 plate, 162.5ms
Speed: 3.1ms preprocess, 162.5ms inference, 2.0ms postprocess pe

In [7]:
from inference_sdk import InferenceHTTPClient
from collections import defaultdict
import base64
import easyocr
import cv2
import math
from ultralytics import YOLO

# Inicializa el lector de EasyOCR
reader = easyocr.Reader(['en'])

# Cargamos el modelo YOLO preentrenado
model = YOLO('yolov8n.pt')

# API Roboflow
apikey_ale = "wwc5TpbiPDQT2hBThzFI"
CLIENT = InferenceHTTPClient(api_url="https://detect.roboflow.com", api_key=apikey_ale)
model_id_roboflow = "license-detector-k1lvk/1"

# Clases relevantes (persona, bicicleta, coche, moto, autobús, camión)
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]

# Función para codificar la imagen a base64
def encode_image_to_base64(image):
    _, buffer = cv2.imencode('.jpg', image)
    jpg_as_text = base64.b64encode(buffer).decode('utf-8')
    return jpg_as_text

# Cargar el video
video_path = 'video.mp4'
vid = cv2.VideoCapture(video_path)

frame_skip = 2  # Procesar un cuadro cada dos
frame_count = 0

while True:
    ret, img = vid.read()

    if not ret:
        break

    # Solo procesar algunos cuadros
    if frame_count % frame_skip != 0:
        frame_count += 1
        continue

    # Reducimos el tamaño para mejorar el rendimiento
    img = cv2.resize(img, (640, 600))

    # Seguimiento en tiempo real para personas y vehículos
    results = model.track(img, persist=True, classes=[0, 1, 2, 3, 5, 7])

    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Extraer coordenadas de la caja
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            class_name = classNames[cls] if cls < len(classNames) else "unknown"
            confidence = math.ceil((box.conf[0] * 100)) / 100

            color = (0, 255, 0) if cls == 0 else (0, 0, 255)

            # Dibujar contenedor y etiqueta
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, f"{class_name} {confidence:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Si es una persona, tapar la cara
            if cls == 0:  # Persona
                face_height = int((y2 - y1) * 0.4)
                cv2.rectangle(img, (x1, y1), (x2, y1 + face_height), (0, 0, 0), -1)

            # Si es un vehículo, intentar detectar la matrícula
            if cls in [2, 3, 5, 7]:  # Vehículos
                plate_img = img[int(y1 + (y2 - y1) * 0.6):y2, x1:x2]

                # Procesar la imagen con Roboflow
                encoded_image = encode_image_to_base64(img)
                response = CLIENT.infer(encoded_image, model_id=model_id_roboflow)
                predictions = response.get("predictions", [])

                for prediction in predictions:
                    # Coordenadas de la matrícula
                    px1 = int(prediction['x'] - prediction['width'] / 2)
                    py1 = int(prediction['y'] - prediction['height'] / 2)
                    px2 = int(prediction['x'] + prediction['width'] / 2)
                    py2 = int(prediction['y'] + prediction['height'] / 2)

                    # Dibuja el contenedor de la matrícula
                    cv2.rectangle(img, (px1, py1), (px2, py2), (255, 0, 0), 2)
                    cv2.putText(img, f"Plate: {round(prediction['confidence'], 2)}", (px1, py1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                    # Extraer la imagen de la matrícula para OCR
                    plate_crop = img[py1:py2, px1:px2]
                    result = reader.readtext(plate_crop)

                    # Mostrar el texto de la matrícula
                    if result:
                        plate_text = result[0][-2]
                        print(f"Matrícula detectada: {plate_text}")
                        cv2.putText(img, plate_text, (px1, py1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

    cv2.imshow('Detección y seguimiento', img)
    frame_count += 1

    # Salir con la tecla ESC
    if cv2.waitKey(1) == 27:  # Ajustar a 1 ms para un mejor rendimiento
        break

vid.release()
cv2.destroyAllWindows()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 608x640 1 car, 1 truck, 159.9ms
Speed: 3.0ms preprocess, 159.9ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 1 truck, 197.0ms
Speed: 9.1ms preprocess, 197.0ms inference, 2.1ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 person, 1 car, 1 bus, 1 truck, 194.8ms
Speed: 5.0ms preprocess, 194.8ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 person, 1 car, 1 truck, 185.7ms
Speed: 3.9ms preprocess, 185.7ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 person, 1 car, 1 truck, 329.0ms
Speed: 9.7ms preprocess, 329.0ms inference, 3.1ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 1 bus, 1 truck, 344.0ms
Speed: 8.4ms preprocess, 344.0ms inference, 3.5ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 person, 1 car, 1 bus, 1 truck, 277.0ms
Speed: 9.2ms preprocess, 277.0ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 640)



In [8]:
from inference_sdk import InferenceHTTPClient
from collections import defaultdict
import base64
import cv2
import easyocr
import math
from ultralytics import YOLO

# Inicializamos el lector de easyocr con soporte para inglés y español
reader = easyocr.Reader(['en', 'es'])
model = YOLO('yolo11n.pt')  # Carga del modelo YOLOv8 nano para detección

CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="wwc5TpbiPDQT2hBThzFI"
)

# Etiquetas de clases relevantes
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]

video_path = 'video.mp4'
vid = cv2.VideoCapture(video_path)
track_history = defaultdict(lambda: [])

# Función para codificar imagen a base64
def encode_image_to_base64(image):
    _, buffer = cv2.imencode('.jpg', image)
    jpg_as_text = base64.b64encode(buffer).decode('utf-8')
    return jpg_as_text

# Función para preprocesar la imagen antes de pasarla a EasyOCR
def preprocess_plate_image(plate_img):
    gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)  # Convertimos a escala de grises
    enhanced = cv2.convertScaleAbs(gray, alpha=1.5, beta=0)  # Aumentamos el contraste
    return enhanced

# Función para detectar matrícula solo si no se ha procesado antes o si hay cambios
def detect_license_plate_local(plate_img):
    processed_img = preprocess_plate_image(plate_img)  # Preprocesamos la imagen
    result = reader.readtext(processed_img)  # Detección con EasyOCR
    for (bbox, text, prob) in result:
        if prob > 0.5:  # Solo mostrar si la confianza es alta
            print(f"Matrícula detectada: {text} con confianza {prob}")
            # Dibujar contenedor y texto
            top_left = tuple(map(int, bbox[0]))
            bottom_right = tuple(map(int, bbox[2]))
            cv2.rectangle(plate_img, top_left, bottom_right, (0, 255, 0), 2)
            cv2.putText(plate_img, text, (top_left[0], top_left[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

while True:
    ret, img = vid.read()

    if ret:
        img = cv2.resize(img, (640, 600))  # Reducimos resolución para mejor rendimiento
        results = model.track(img, persist=True, classes=[0, 1, 2, 3, 5, 7])

        for r in results:
            boxes = r.boxes

            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                if box.id is not None:
                    track_id = str(int(box.id[0].tolist()))
                else:
                    track_id = ''

                confidence = math.ceil((box.conf[0] * 100)) / 100
                cls = int(box.cls[0])
                class_name = classNames[cls] if cls < len(classNames) else "unknown"

                color = (0, 255, 0) if cls == 0 else (0, 0, 255)  # Verde para personas, rojo para vehículos
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, f"{track_id} {class_name} {confidence:.2f}", 
                            (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                # Si es un vehículo, realizar la detección de matrícula
                if cls in [2, 3, 5, 7]:  # Solo vehículos
                    if track_id not in track_history or len(track_history[track_id]) == 0:
                        # Detectamos solo si es un vehículo nuevo o no ha sido procesado
                        plate_img = img[y1:y2, x1:x2]
                        detect_license_plate_local(plate_img)
                        track_history[track_id].append("processed")

        cv2.imshow('Detección y seguimiento', img)
    else:
        break

    if cv2.waitKey(20) == 27:  # Presiona 'Esc' para cerrar la ventana
        break

vid.release()
cv2.destroyAllWindows()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 608x640 1 person, 1 car, 220.9ms
Speed: 3.0ms preprocess, 220.9ms inference, 2.6ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 191.3ms
Speed: 2.0ms preprocess, 191.3ms inference, 1.6ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 person, 1 car, 233.1ms
Speed: 2.0ms preprocess, 233.1ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 person, 1 car, 235.1ms
Speed: 2.0ms preprocess, 235.1ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 2 persons, 1 car, 247.0ms
Speed: 5.3ms preprocess, 247.0ms inference, 3.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 246.0ms
Speed: 2.6ms preprocess, 246.0ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 248.0ms
Speed: 3.0ms preprocess, 248.0ms inference, 4.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 193.9ms
Speed: 4.0ms preprocess, 193.9ms inference, 1.0ms postpro

In [9]:
from collections import defaultdict
import cv2
import easyocr
import math
from ultralytics import YOLO

# Inicialización del lector de OCR y el modelo YOLOv8
reader = easyocr.Reader(['en'])
model = YOLO('yolov8n.pt')

# Etiquetas de clases relevantes
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]

# Ruta del video y captura de frames
video_path = 'video.mp4'
vid = cv2.VideoCapture(video_path)
track_history = defaultdict(lambda: False)  # Almacena si una matrícula ya fue procesada

# Función para detectar matrículas
def detect_license_plate_local(plate_img):
    result = reader.readtext(plate_img)
    for bbox, text, prob in result:
        if prob > 0.5:  # Solo mostrar si la confianza es alta
            print(f"Matrícula detectada: {text} con confianza {prob}")
            # Dibujar rectángulo y texto
            top_left = tuple(map(int, bbox[0]))
            bottom_right = tuple(map(int, bbox[2]))
            cv2.rectangle(plate_img, top_left, bottom_right, (0, 255, 0), 2)
            cv2.putText(plate_img, text, (top_left[0], top_left[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Bucle principal para procesar el video
while vid.isOpened():
    ret, img = vid.read()

    if not ret:
        break

    img = cv2.resize(img, (640, 600))  # Reducción de resolución para procesamiento más rápido
    results = model.track(img, persist=True, classes=[0, 1, 2, 3, 5, 7])

    # Procesar detecciones
    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            track_id = str(int(box.id[0].tolist())) if box.id is not None else ''
            confidence = box.conf[0].item()
            cls = int(box.cls[0])
            class_name = classNames[cls] if cls < len(classNames) else "unknown"

            # Color: verde para personas, rojo para vehículos
            color = (0, 255, 0) if cls == 0 else (0, 0, 255)
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, f"{track_id} {class_name} {confidence:.2f}", 
                        (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Detección de matrícula para vehículos no procesados previamente
            if cls in [2, 3, 5, 7]:  # Solo vehículos
                if not track_history[track_id]:  # Solo procesar vehículos nuevos
                    plate_img = img[y1:y2, x1:x2]  # Recortar la imagen del vehículo
                    detect_license_plate_local(plate_img)
                    track_history[track_id] = True  # Marcar como procesado

    # Mostrar la imagen procesada
    cv2.imshow('Detección y seguimiento', img)

    # Salir si se presiona la tecla 'Esc'
    if cv2.waitKey(20) == 27:
        break

vid.release()
cv2.destroyAllWindows()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 608x640 1 car, 1 truck, 97.1ms
Speed: 2.0ms preprocess, 97.1ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 1 bus, 1 truck, 89.8ms
Speed: 2.0ms preprocess, 89.8ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 1 truck, 85.4ms
Speed: 2.5ms preprocess, 85.4ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 person, 1 car, 1 truck, 89.0ms
Speed: 1.8ms preprocess, 89.0ms inference, 2.4ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 person, 1 car, 1 bus, 1 truck, 85.1ms
Speed: 2.1ms preprocess, 85.1ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 1 truck, 76.5ms
Speed: 2.4ms preprocess, 76.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 person, 1 car, 1 truck, 88.4ms
Speed: 2.0ms preprocess, 88.4ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 2 persons, 1 car, 1 

### 1 modelo posible

In [10]:

from inference_sdk import InferenceHTTPClient
from collections import defaultdict
import base64


reader = easyocr.Reader(['en']) # Inicializamos el lector de easyocr

# Carga del modelo preentrenado YOLO (nano)
model = YOLO('yolov8n.pt')
apikey_heliot="wwc5TpbiPDQT2hBThzFI"
apikey_ale="aLeqgEj29wrZMNq8Rg4X"

model_id_heliot = "license-detector-k1lvk/1"
model_id_heliot = "license-plate-ojt1x/1"
model_id_alejandro="cochecitos/1"

CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key= apikey_ale
)

  
# Etiqueta de las distintas clases relevantes 
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]

video_path = 'video.mp4'
vid = cv2.VideoCapture(video_path)
track_history = defaultdict(lambda: [])


# Función para codificar la imagen a base64
def encode_image_to_base64(image):
    _, buffer = cv2.imencode('.jpg', image)
    jpg_as_text = base64.b64encode(buffer).decode('utf-8')
    return jpg_as_text



while True:
    ret, img = vid.read()
  
    # Verificamos si hay una imagen válida
    if ret:
        # Reducimos la resolución de la imagen para mejorar la velocidad
        img = cv2.resize(img, (640, 600))

        # Seguimiento en tiempo real para clases relevantes (personas y vehículos)
        results = model.track(img, persist=True, classes=[0, 1, 2, 3, 5, 7])

        # Iteramos sobre cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Extraemos las coordenadas de la caja delimitadora
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Etiqueta de seguimiento
                if box.id is not None:
                    track_id = str(int(box.id[0].tolist()))
                else:
                    track_id = ''

                # Nivel de confianza
                confidence = math.ceil((box.conf[0] * 100)) / 100

                # Clase detectada
                cls = int(box.cls[0])
                class_name = classNames[cls] if cls < len(classNames) else "unknown"

                color = (0, 255, 0) if cls == 0 else (0, 0, 255)  # Verde para personas, rojo para vehículos

                # Dibujamos el contenedor y la etiqueta de clase
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, f"{track_id} {class_name} {confidence:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                # Si es una persona, tapamos la cara
                if cls == 0: 
                    face_height = int((y2 - y1) * 0.4)  # Asumimos que la cara ocupa el 40% superior del cuerpo
                    cv2.rectangle(img, (x1, y1), (x2, y1 + face_height), (0, 0, 0), -1)  # Rectángulo negro

                # Si es un vehículo, intentar detectar la matrícula
                if cls in [2, 3, 5, 7]: 
                    # Enfocamos en la parte baja del vehículo
                    plate_img = img[int(y1 + (y2 - y1) * 0.6):y2, x1:x2]

                    # Preprocesamos la imagen de la matrícula
                    gray_plate_img = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
                    enhanced_plate_img = cv2.equalizeHist(gray_plate_img)  # Aumentamos el contraste
                    _, binary_plate_img = cv2.threshold(enhanced_plate_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                    # ROBOFLOW INSERTAR
                    
                     # Hacer la inferencia en el fotograma usando Roboflow
                    encoded_image = encode_image_to_base64(img)
        
                    # Hacer la inferencia en el fotograma usando Roboflow
                    response = CLIENT.infer(
                        encoded_image,  # La imagen codificada como base64
                        model_id="license-detector-k1lvk/1"
                    )
                    predictions = response.get("predictions", [])
                      # Para cada predicción en el fotograma
                    for prediction in predictions:
                        # Coordenadas de la caja de detección
                        x1 = int(prediction['x'] - prediction['width'] / 2)
                        y1 = int(prediction['y'] - prediction['height'] / 2)
                        x2 = int(prediction['x'] + prediction['width'] / 2)
                        y2 = int(prediction['y'] + prediction['height'] / 2)

                        # Clase de la detección
                        cls = prediction.get('class', '')
                        if cls in classNames:
                            cls_index = classNames.index(cls)
                        else:
                            cls_index = -1  # No se reconoce la clase

                        # Confianza
                        confidence = round(prediction['confidence'], 2)
                        print(f"Confianza ---> {confidence}")
                        print(f"Clase ---> {cls}")

                        # Dibuja el contenedor y clase
                        cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
                        cv2.putText(img, f"{cls} {confidence}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        cv2.imshow('Detección y super seguimiento', img)
    else:
        break 

    # Salimos del bucle si se presiona ESC
    if cv2.waitKey(20) == 27:
        break

vid.release()
cv2.destroyAllWindows()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 608x640 1 car, 1 truck, 82.6ms
Speed: 1.0ms preprocess, 82.6ms inference, 1.3ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 1 bus, 1 truck, 116.9ms
Speed: 3.1ms preprocess, 116.9ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 1 truck, 102.8ms
Speed: 2.5ms preprocess, 102.8ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 person, 1 car, 1 truck, 101.5ms
Speed: 3.2ms preprocess, 101.5ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 person, 1 car, 1 bus, 1 truck, 100.4ms
Speed: 2.0ms preprocess, 100.4ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 car, 1 truck, 102.8ms
Speed: 3.0ms preprocess, 102.8ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 person, 1 car, 1 truck, 98.0ms
Speed: 3.2ms preprocess, 98.0ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)


### 2 modelo posible

In [9]:
# Tamaño del video
desired_size = (800, 500)

model = YOLO('modelo.pt')  # Modelo de yolo preentrenado

pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

def preprocess_plate(plate):
    """Preprocesar la imagen de la matrícula para mejorar la detección de texto."""
    gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    # Aplicamos umbral adaptativo para resaltar caracteres
    processed_plate = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    return processed_plate

def show_image(window_name, image):
    resized_image = cv2.resize(image, desired_size)
    cv2.imshow(window_name, resized_image)

# Leemos el video
video_path = "video.mp4"
cap = cv2.VideoCapture(video_path)

# Verificamos si el video se abrió correctamente
if not cap.isOpened():
    print("Error al abrir el video")
    exit()

while True:
    ret, frame = cap.read()

    # Si no se leen más cuadros, salimos
    if not ret:
        break

    # Detectamos matrículas con el modelo
    results = model(frame, verbose=False)

    # Iteramos sobre cada detección en el resultado
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Coordenadas del contenedor
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # Convertimos a valores enteros

            # Extraemos solo la matrícula
            plate = frame[y1:y2, x1:x2]

            # Preprocesar la imagen de la matrícula
            processed_plate = preprocess_plate(plate)

            # Recuadro de la matrícula
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

            # Expresión regular para detectar la matrícula "0000 AAA"
            plate_pattern = re.compile(r'^\d{4}[A-Z]{3}$')

            # Utilizamos pytesseract para la detección de texto en la imagen preprocesada
            text = pytesseract.image_to_string(processed_plate, config='--psm 8', output_type=Output.STRING)

            # Limpiamos el texto detectado
            text = re.sub(r'[^A-Za-z0-9]', '', text).strip()

            # Verificamos si el texto cumple con el formato de matrícula
            if plate_pattern.match(text):
                print("Matrícula detectada:", text)
            else:
                print("Texto detectado (no coincide con patrón de matrícula):", text)

    # Mostramos el video
    show_image("Super mega detector", frame)

    # Salimos si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar el video
cap.release()
cv2.destroyAllWindows()


Texto detectado (no coincide con patrón de matrícula): ie
Texto detectado (no coincide con patrón de matrícula): fal
Texto detectado (no coincide con patrón de matrícula): ERT
Texto detectado (no coincide con patrón de matrícula): 2GKa
Texto detectado (no coincide con patrón de matrícula): 2867Ku
Texto detectado (no coincide con patrón de matrícula): N2867KH
Texto detectado (no coincide con patrón de matrícula): N2857KHZ
Matrícula detectada: 2867XHZ
Texto detectado (no coincide con patrón de matrícula): 32867KHZ
Matrícula detectada: 2867KHZ
Matrícula detectada: 2867KHZ
Texto detectado (no coincide con patrón de matrícula): 2887XZ
Texto detectado (no coincide con patrón de matrícula): 42867KHZ
Texto detectado (no coincide con patrón de matrícula): 42867KH
Texto detectado (no coincide con patrón de matrícula): 42867KH
Texto detectado (no coincide con patrón de matrícula): 42867KHZ
Texto detectado (no coincide con patrón de matrícula): I2867KHZ
Texto detectado (no coincide con patrón de m

In [7]:
import cv2
import re
import csv
import pytesseract
from ultralytics import YOLO
from pytesseract import Output
import numpy as np

# Tamaño del video
desired_size = (800, 500)

# Cargamos el modelo YOLO preentrenado
model = YOLO('yolov8n.pt')  # Modelo YOLOv8 Nano preentrenado

pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

def preprocess_plate(plate):
    """Preprocesar la imagen de la matrícula para mejorar la detección de texto."""
    gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    processed_plate = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    return processed_plate

def show_image(window_name, image):
    resized_image = cv2.resize(image, desired_size)
    cv2.imshow(window_name, resized_image)

# Leemos el video
video_path = "video.mp4"
cap = cv2.VideoCapture(video_path)

# Verificamos si el video se abrió correctamente
if not cap.isOpened():
    print("Error al abrir el video")
    exit()

# Definir las clases relevantes
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]

# Inicializamos el archivo CSV
csv_file = open("detected_objects.csv", mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['frame', 'object_type', 'confidence', 'tracking_id', 'x1', 'y1', 'x2', 'y2', 'license_plate', 'plate_confidence', 'mx1', 'my1', 'mx2', 'my2', 'plate_text'])

# Inicializamos el objeto para grabar el video con los resultados
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Detección de objetos en el frame
    results = model(frame, verbose=False)

    # Contadores de clases
    class_counts = {class_name: 0 for class_name in classNames}

    # Procesamos los resultados de detección de YOLO
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Coordenadas del contenedor
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # Convertimos a valores enteros
            confidence = box.conf[0].item()  # Confianza de la detección
            class_id = int(box.cls[0].item())  # ID de la clase

            # Verificar si el ID de clase está dentro de los límites de classNames
            if class_id < len(classNames):
                class_name = classNames[class_id]
            else:
                continue  # Saltar esta detección si el ID está fuera de rango

            # Contamos el objeto detectado
            class_counts[class_name] += 1

            tracking_id = box.track_id if hasattr(box, 'track_id') else None  # Seguimiento si está disponible

            # Recuadro de la detección
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f"{class_name} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

            # Si es un vehículo, aplicamos pytesseract para leer la matrícula
            if class_name in ["car", "motorbike", "bus", "truck"]:
                plate = frame[y1:y2, x1:x2]
                processed_plate = preprocess_plate(plate)

                # Utilizamos pytesseract para la detección de texto
                plate_text = pytesseract.image_to_string(processed_plate, config='--psm 8', output_type=Output.STRING)

                # Limpiamos el texto detectado
                plate_text = re.sub(r'[^A-Za-z0-9]', '', plate_text).strip()

                # Verificamos si el texto cumple con el formato de matrícula
                plate_pattern = re.compile(r'^\d{4}[A-Z]{3}$')
                plate_detected = plate_pattern.match(plate_text)

                # Guardamos la información de la matrícula si es válida
                if plate_detected:
                    print("Matrícula detectada:", plate_text)
                    mx1, my1, mx2, my2 = x1, y1, x2, y2  # Coordenadas de la matrícula
                    plate_conf = confidence
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

                    # Escribir en el CSV
                    csv_writer.writerow([frame_count, class_name, confidence, tracking_id, x1, y1, x2, y2, plate_text, confidence, mx1, my1, mx2, my2, plate_text])

                else:
                    print("Texto detectado (no coincide con patrón de matrícula):", plate_text)

    # Mostramos el video
    show_image("Super mega detector", frame)

    # Salimos si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberamos los recursos
cap.release()
csv_file.close()
cv2.destroyAllWindows()


KeyboardInterrupt: 

### 3 posible modelo

In [12]:

# Cargamos nuestro modelo YOLO preentrenado
model = YOLO('best.pt')

classNames = ["Super matricula"]  # Clase de matrícula

# Inicializa el lector de EasyOCR
reader = easyocr.Reader(['es'])

# Ruta del video
video_path = 'video.mp4'

# Abrimos
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, img = cap.read()
    
    if not ret:
        break  # Salimos del bucle si se acaba el video

    results = model(img, stream=True)

    # Para cada detección
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Tomamos el contenedor
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            # Confianza
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confianza: ", confidence)

            # Clase
            cls = int(box.cls[0])
            print("Nombre de la clase: ", classNames[0])

            # Dibujamos el contenedor
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f"{classNames[0]} {confidence}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Recortamos el área de interés (matrícula)
            img_cropped = img[y1:y2, x1:x2]
            if img_cropped.size > 0:
                # Reconocimiento de una imagen
                result = reader.readtext(img_cropped, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')
                output = ""

                if len(result) >= 1:
                    first = result[0][1]
                else:
                    first = ""

                if len(result) >= 2:
                    second = result[1][1]
                else:
                    second = ""

                if first.isnumeric():
                    output = first + " " + second
                else:
                    output = second + " " + first

                print("La matrícula detectada es:", output)

    cv2.imshow('Detector increible de Matriculas', img)

    # Salimos del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


KeyboardInterrupt: 

In [4]:
import cv2
import math
import re
import csv
import easyocr
from ultralytics import YOLO

# Cargamos el modelo YOLO para detección de vehículos y personas
detection_model = YOLO('yolov8n.pt')

# Cargamos el modelo YOLO preentrenado para detectar matrículas
license_plate_model = YOLO('best.pt')

# Clases de interés (personas y vehículos)
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]

# Inicializamos el lector de EasyOCR
reader = easyocr.Reader(['es'])

# Ruta del video
video_path = 'video.mp4'
cap = cv2.VideoCapture(video_path)

# Inicializamos el archivo CSV
csv_file = open("detected_objects.csv", mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['frame', 'object_type', 'confidence', 'tracking_id', 'x1', 'y1', 'x2', 'y2', 'license_plate', 'plate_confidence', 'direction'])

# Inicializamos el objeto para grabar el video con los resultados
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

frame_count = 0
directions = {"left": 0, "right": 0, "up": 0, "down": 0}  # Contadores de dirección

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break  # Salimos del bucle si se acaba el video

    frame_count += 1
    detection_results = detection_model(img, stream=True)  # Detección de vehículos y personas

    # Procesamos los resultados de detección de YOLO
    for r in detection_results:
        boxes = r.boxes

        for box in boxes:
            # Tomamos las coordenadas del contenedor
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = math.ceil((box.conf[0] * 100)) / 100  # Confianza
            cls = int(box.cls[0])  # Índice de clase

            if cls < len(classNames):
                class_name = classNames[cls]

                # Anonimizar las personas
                if class_name == "person":
                    # Aplicamos un desenfoque a la región de la persona
                    person_roi = img[y1:y2, x1:x2]
                    blurred_person = cv2.GaussianBlur(person_roi, (51, 51), 0)
                    img[y1:y2, x1:x2] = blurred_person
                    direction = "left" if x2 < img.shape[1] // 2 else "right"
                    directions[direction] += 1
                elif class_name in ["car", "motorbike", "bus", "truck"]:
                    direction = "left" if x2 < img.shape[1] // 2 else "right"
                    directions[direction] += 1

                # Seguimiento (si está disponible)
                tracking_id = box.track_id if hasattr(box, 'track_id') else None

                # Dibujamos el rectángulo de la detección
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, f"{class_name} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                # Si es un vehículo, aplicamos el modelo de matrículas
                plate_text = ""
                if class_name in ["car", "motorbike", "bus", "truck"]:
                    img_cropped = img[y1:y2, x1:x2]
                    if img_cropped.size > 0:
                        # Detectamos la matrícula con el modelo entrenado para matrículas
                        license_plate_results = license_plate_model(img_cropped, stream=True)

                        # Procesamos el generador directamente
                        for plate in license_plate_results:
                            plate_boxes = plate.boxes
                            for plate_box in plate_boxes:
                                # Extraemos las coordenadas de la matrícula
                                px1, py1, px2, py2 = map(int, plate_box.xyxy[0])
                                img_plate_cropped = img_cropped[py1:py2, px1:px2]
                                
                                # Leemos el texto de la matrícula usando EasyOCR
                                result = reader.readtext(img_plate_cropped, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')
                                
                                # Procesamos el texto detectado
                                if len(result) >= 1:
                                    plate_text = re.sub(r'[^A-Za-z0-9]', '', result[0][1]).strip()  # Limpiamos el texto

                                # Dibujamos el texto de la matrícula en la imagen
                                if plate_text:
                                    cv2.putText(img, plate_text, (x1, y2 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                                    print("Matrícula detectada:", plate_text)
                                else:
                                    print("No se detectó matrícula.")

                # Escribimos los datos en el archivo CSV
                csv_writer.writerow([frame_count, class_name, confidence, tracking_id, x1, y1, x2, y2, plate_text, confidence, direction])

    # Guardamos el frame procesado en el video de salida
    out.write(img)

    # Mostramos el video con las detecciones
    cv2.imshow('Detector de personas y vehículos', img)

    # Salimos si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberamos los recursos
cap.release()
out.release()
csv_file.close()
cv2.destroyAllWindows()

# Imprimimos el conteo de direcciones
print("Direcciones detectadas:")
for direction, count in directions.items():
    print(f"{direction}: {count}")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 384x640 1 person, 1 car, 164.3ms

0: 544x640 (no detections), 165.5ms
Speed: 3.0ms preprocess, 165.5ms inference, 1.1ms postprocess per image at shape (1, 3, 544, 640)
Speed: 4.0ms preprocess, 164.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 car, 170.5ms

0: 544x640 (no detections), 199.0ms
Speed: 4.2ms preprocess, 199.0ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 640)
Speed: 2.0ms preprocess, 170.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 152.1ms

0: 544x640 (no detections), 156.4ms
Speed: 2.3ms preprocess, 156.4ms inference, 1.1ms postprocess per image at shape (1, 3, 544, 640)
Speed: 2.0ms preprocess, 152.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 2 cars, 187.8ms

0: 544x640 (no detections), 155.8ms
Speed: 2.3ms preprocess, 155.8ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 640)

0: 640x384 (no

### Otro posible modelo usando pytesseract para la deteccion de matriculas

In [ ]:
import cv2
import math
import re
import csv
import pytesseract
from ultralytics import YOLO

# Configuramos la ruta de Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

# Cargamos el modelo YOLO para detección de vehículos y personas
detection_model = YOLO('yolov8n.pt')

# Cargamos el modelo YOLO preentrenado para detectar matrículas
license_plate_model = YOLO('modelo.pt')

# Clases de interés (personas y vehículos)
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]

# Ruta del video
video_path = 'video.mp4'
cap = cv2.VideoCapture(video_path)

# Inicializamos el archivo CSV a crear
csv_file = open("detected_objects_pytesseract.csv", mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['frame', 'object_type', 'confidence', 'tracking_id', 'x1', 'y1', 'x2', 'y2', 'license_plate', 'plate_confidence', 'direction'])

# Inicializamos el objeto para grabar el video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video_pytesseract.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

frame_count = 0
directions = {"left": 0, "right": 0, "up": 0, "down": 0}  # Contadores de direccion

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break  # Salimos del bucle si se acaba el video

    frame_count += 1
    detection_results = detection_model(img, stream=True)  # Detección de vehículos y personas

    # Procesamos los resultados de detección de YOLO
    for r in detection_results:
        boxes = r.boxes

        for box in boxes:
            # Tomamos las coordenadas del contenedor
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = math.ceil((box.conf[0] * 100)) / 100  # Confianza
            cls = int(box.cls[0])  # Índice de clase

            if cls < len(classNames):
                class_name = classNames[cls]

                # Anonimizar las personas
                if class_name == "person":
                    # Aplicamos un desenfoque a la región de la persona
                    person_roi = img[y1:y2, x1:x2]
                    blurred_person = cv2.GaussianBlur(person_roi, (51, 51), 0)
                    img[y1:y2, x1:x2] = blurred_person
                    direction = "left" if x2 < img.shape[1] // 2 else "right"
                    directions[direction] += 1

                elif class_name in ["car", "motorbike", "bus", "truck"]:
                    direction = "left" if x2 < img.shape[1] // 2 else "right"
                    directions[direction] += 1

                # Seguimiento (si está disponible)
                tracking_id = box.track_id if hasattr(box, 'track_id') else None

                # Dibujamos el rectángulo de la detección
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, f"{class_name} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                # Si es un vehículo, aplicamos el modelo de matrículas
                plate_text = ""
                if class_name in ["car", "motorbike", "bus", "truck"]:
                    
                    img_cropped = img[y1:y2, x1:x2] # Nos quedamos con los pixeles reconocidos como vehiculo ROI

                    if img_cropped.size > 0:
                        # Aplicamos el modelo para detectar matrículas
                        license_plate_results = license_plate_model(img_cropped, stream=True)

                        # Procesamos el generador directamente
                        for plate in license_plate_results:
                            plate_boxes = plate.boxes
                            for plate_box in plate_boxes:
                                # Extraemos las coordenadas de la matrícula
                                px1, py1, px2, py2 = map(int, plate_box.xyxy[0])
                                img_plate_cropped = img_cropped[py1:py2, px1:px2]
                                
                                # Leemos el texto de la matrícula usando pytesseract
                                text = pytesseract.image_to_string(img_plate_cropped, config='--psm 8 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
                                plate_text = re.sub(r'[^A-Za-z0-9]', '', text).strip()  # Limpiamos el texto

                                # Dibujamos el texto de la matrícula en la imagen
                                if plate_text:
                                    cv2.putText(img, plate_text, (x1, y2 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                                    print("Matrícula detectada:", plate_text)
                                else:
                                    print("No se detectó matrícula.")

                # Escribimos los datos en el archivo CSV
                csv_writer.writerow([frame_count, class_name, confidence, tracking_id, x1, y1, x2, y2, plate_text, confidence, direction])

    # Guardamos el frame procesado en el video de salida
    out.write(img)

    # Mostramos el video con las detecciones
    cv2.imshow('Detector de personas y vehículos', img)

    # Salimos si se presiona "q"
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
csv_file.close()
cv2.destroyAllWindows()

print("Direcciones detectadas:")
for direction, count in directions.items():
    print(f"{direction}: {count}")

Como podemos ver este 2 modelo funciona como el anterior aunque falla mas en seguir las matriculas, y predecirlas sin fallar de manera correcta.